In [16]:
import undetected_chromedriver as uc
# from selenium import webdriver
from time import sleep as time_sleep
from bs4 import BeautifulSoup as BS
import requests
import subprocess
from config import headers, cookies
from pprint import pprint
import lxml
import pandas as pd

In [17]:
url_list = '''
https://novelbin.com/b/the-mech-touch
https://novelbin.com/b/shadow-slave
'''.strip().split('\n')


url = url_list[0]

host = '/'.join(url.split('/', maxsplit=3)[:3])
name_title = url.split('/')[-1]
print(name_title)

chapters_link = f'{host}/ajax/chapter-option?novelId={name_title}&currentChapterId'
print(chapters_link)

the-mech-touch
https://novelbin.com/ajax/chapter-option?novelId=the-mech-touch&currentChapterId


In [18]:
list_links = list()

tt = requests.get(chapters_link, headers=headers, cookies=cookies)
links_value = BS(tt.text, 'lxml')
tb_list = links_value.find_all('option')
elem : BS
for elem in tb_list:
    list_links.append(elem.get('value'))
print('Всего ссылок',len(list_links))
print('Последняя ссылка:')
print(list_links[-1])

Всего ссылок 5580
Последняя ссылка:
https://novelbin.com/b/the-mech-touch/chapter-5580-divergent-paths


In [21]:
driver = uc.Chrome()
# driver = webdriver.Chrome()

driver.get(host)
time_sleep(1)
driver.get(url)
for _ in range(200):
    page = driver.page_source
    if not 'cloudflare.com' in page:
        break
    else:
        time_sleep(1)
else:
    driver.quit()
print('yes')

yes


In [23]:
book_eng = list()

for link in list_links[400:600]:
    driver.get(link)
    for _ in range(200):
        page = driver.page_source
        if not 'cloudflare.com' in page:
            break
        else:
            time_sleep(1)
    else:
        driver.quit()
    time_sleep(0.5)
    soup = BS(page, 'html.parser')
    novel_title = soup.find(attrs={'class':'novel-title'})
    content = soup.find(attrs={'id':'chr-content'})
    
    contents = content.find_all('p')
    
    content_list = list()
    el: BS
    for el in contents:
        content_list.append(el.get_text())
    text = '\n'.join(content_list)
    book_eng.append(text)
len(book_eng)

200

In [24]:
driver.quit()

In [54]:
book_eng_full = '\n'.join(book_eng)
a = pd.Series(book_eng_full.split('\n')).str.strip()
b = a.value_counts(dropna=False).to_frame().query('count > 4')
b

,count
,11869
You can trace the roots of this content at n0v@lbin,64
The source of this content can be linked to n0v𝓮lb𝕚n•,32
This content originates from n0v3lbin•,26
Trace the roots of this material to novelb!n•,22
The original content can be pinpointed to n0v3lbin•,15
This material is derived from n0v£lbin•,13
The source of this material is novelbin★,10
Explore the genesis of this content at n𝒐v@lbin★,8
You'll find the origin of this content at n0velb!n•,6


In [59]:
c = '\n'.join(a.drop(index=a[a.isin(b.index)].index, errors='ignore').values)

In [8]:
# # clean text:
# test = book_eng_full.split('\n')
# test2 = list()
# for row in test:
#     if 'The source of this content is' in row:
#         continue
#     test2.append(row)
# book_eng_full = '\n'.join(test2)

# book_eng_full = book_eng_full.replace('6054d257f56b520818c0fb96','')
# book_eng_full = book_eng_full.replace('                  Ads by PubFuture','')
# book_eng_full = book_eng_full.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')

In [60]:
with open(
    file=f'{name_title}.txt',
    mode='w',
    encoding='utf-8'
) as file:
    # file.write(book_eng_full)
    file.write(c)

In [61]:
with open(
    file=f'{name_title}.txt',
    encoding='utf-8'
) as file:
    book_eng_full = file.read()
# book_eng_full

In [62]:
from yandexfreetranslate import YandexFreeTranslate, YandexFreeTranslateError
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm
from urllib.error import URLError

def _worker(que: tuple) -> str:
    for _ in range(10):
        queue, pbar = que
        if not queue:
            pbar.update(1)
            return ''
        yt = YandexFreeTranslate(api = "ios")
        try:
            text = yt.translate("en", "ru", queue)
        except TimeoutError:
            continue
        except URLError:
            time_sleep(1)
            continue
            
        except YandexFreeTranslateError:
            time_sleep(1)
            continue
            
        # text = text.replace('/n', '')
        pbar.update(1)
        return text
    raise Exception()

def run_translate(lst: list[str])-> list[str]:
    with tqdm(total=len(lst) ) as pbar:
        l_b = list(zip(lst, ( pbar for _ in range(len(lst)))))
        with ThreadPool(30) as pool:
            workreturn = pool.map(_worker, l_b)
    return workreturn

lst = book_eng_full.split('\n')

text_rus = run_translate(lst)

text_rus = '\n'.join(text_rus)

with open(
    file=f'{name_title}_rus.txt',
    mode='w',
    encoding='utf-8'
) as file:
    file.write(text_rus)

  0%|          | 0/12259 [00:00<?, ?it/s]